In [1]:
# import Libraries
import yt_dlp
from pydub import AudioSegment
from src.youtube_downloader import YouTubeDownloader
# pip install yt-dlp
# pip install pydub
# pip install moviepy

/opt/anaconda3/lib/python3.11/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [ ]:
# Initialize downloader
downloader = YouTubeDownloader()

In [1]:
from yt_dlp import YoutubeDL
import sys
import os

def download_youtube_video(url, format_id=None):
    # Ensure ffmpeg is in PATH
    if '/usr/local/bin' not in os.environ['PATH']:
        os.environ['PATH'] += ':/usr/local/bin'

    # Configuración para yt-dlp
    ydl_opts = {
        'format': format_id if format_id else 'bestvideo[ext=mp4][height<=1080]+bestaudio[ext=m4a]/best[ext=mp4]/mp4',
        'outtmpl': '%(title)s.%(ext)s',
        'quiet': False,
        'no_warnings': False,
        'cookiesfrombrowser': ('chrome',),
        'extractor_retries': 5,
        'fragment_retries': 10,
        'retries': 10,
        'file_access_retries': 5,
        'ffmpeg_location': '/usr/local/bin/ffmpeg',  # Explicitly set ffmpeg path
        'http_headers': {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
        },
        'merge_output_format': 'mp4',
        'postprocessors': [{
            'key': 'FFmpegVideoConvertor',
            'preferedformat': 'mp4',
        }]
    }

    # Verify ffmpeg installation
    def check_ffmpeg():
        try:
            import subprocess
            result = subprocess.run(['ffmpeg', '-version'], 
                                  stdout=subprocess.PIPE, 
                                  stderr=subprocess.PIPE)
            return result.returncode == 0
        except Exception:
            return False

    if not check_ffmpeg():
        print('Error: ffmpeg is not properly installed or not found in PATH')
        print('Please ensure ffmpeg is installed and accessible')
        return False

    def try_different_formats(ydl, url):
        formats_to_try = [
            'bestvideo[ext=mp4][height<=1080]+bestaudio[ext=m4a]',
            'best[ext=mp4]',
            '22',
            '18',
            'worstvideo[ext=mp4]+worstaudio[ext=m4a]'
        ]
        
        for fmt in formats_to_try:
            try:
                ydl_opts['format'] = fmt
                with YoutubeDL(ydl_opts) as ydl:
                    print(f'\nTrying format: {fmt}')
                    info = ydl.extract_info(url, download=False)
                    formats = info.get('formats', [])
                    
                    # Filter for MP4 formats only
                    mp4_formats = [f for f in formats if f.get('ext') == 'mp4' or 
                                 (f.get('vcodec') != 'none' and f.get('acodec') != 'none')]
                    
                    if mp4_formats:
                        return True, info
            except Exception as e:
                if '403' not in str(e):  # Only continue if it's not a 403 error
                    raise e
                print(f'Format {fmt} not available, trying next...')
        return False, None

    try:
        # Try to get available formats first
        success, info = try_different_formats(YoutubeDL(ydl_opts), url)
        
        if not success:
            print('Could not find any suitable MP4 format. Trying alternative method...')
            # Try with minimal options but still with ffmpeg path
            ydl_opts = {
                'format': '18/best[ext=mp4]',
                'outtmpl': '%(title)s.%(ext)s',
                'quiet': False,
                'merge_output_format': 'mp4',
                'ffmpeg_location': '/usr/local/bin/ffmpeg'
            }
        
        print(f'\nDescargando: {url}')
        with YoutubeDL(ydl_opts) as ydl:
            result = ydl.download([url])
            
            if result == 0:
                print('Descarga completada exitosamente.')
                # Verify the downloaded file is MP4
                if info and info.get('title'):
                    filename = f"{info['title']}.mp4"
                    if os.path.exists(filename):
                        print(f'Archivo guardado como: {filename}')
                return True
            else:
                print('Error durante la descarga.')
                return False

    except Exception as e:
        print(f'Error al descargar el video: {e}')
        if 'ffmpeg not installed' in str(e).lower():
            print('\nError: ffmpeg is required but not properly configured.')
            print('Please ensure ffmpeg is installed at /usr/local/bin/ffmpeg')
        elif '403' in str(e):
            print('\nError: Access forbidden. This might be due to:')
            print('1. Region restrictions')
            print('2. Age restrictions')
            print('3. Private video')
            print('4. YouTube bot detection')
            print('\nTry:')
            print('1. Logging into YouTube in your browser')
            print('2. Using a VPN')
            print('3. Checking if the video is available in your region')
        return False

# Example: Download a video
url = input('Enter YouTube URL: ')
download_youtube_video(url)


Trying format: bestvideo[ext=mp4][height<=1080]+bestaudio[ext=m4a]
Extracting cookies from chrome
Extracted 3398 cookies from chrome
[youtube] Extracting URL: https://youtube.com/shorts/moH5DZcmv6E?si=v3U7RCQm7qN0doic
[youtube] moH5DZcmv6E: Downloading webpage


[youtube] moH5DZcmv6E: Downloading tv client config
[youtube] moH5DZcmv6E: Downloading player 093288cd-tv
[youtube] moH5DZcmv6E: Downloading tv player API JSON
[youtube] moH5DZcmv6E: Downloading web player API JSON
[youtube] moH5DZcmv6E: Downloading initial data API JSON

Descargando: https://youtube.com/shorts/moH5DZcmv6E?si=v3U7RCQm7qN0doic
Extracting cookies from chrome
Extracted 3398 cookies from chrome
[youtube] Extracting URL: https://youtube.com/shorts/moH5DZcmv6E?si=v3U7RCQm7qN0doic
[youtube] moH5DZcmv6E: Downloading webpage


[youtube] moH5DZcmv6E: Downloading tv client config
[youtube] moH5DZcmv6E: Downloading tv player API JSON
[youtube] moH5DZcmv6E: Downloading web player API JSON
[youtube] moH5DZcmv6E: Downloading initial data API JSON
[info] moH5DZcmv6E: Downloading 1 format(s): 788+140
[download] Destination: Por qué ellos tienen éxito y tú no？ #desarrollopersonal #inspiracion #motivacion.f788.mp4
[download] 100% of    1.42MiB in 00:00:01 at 1.38MiB/s   
[download] Destination: Por qué ellos tienen éxito y tú no？ #desarrollopersonal #inspiracion #motivacion.f140.m4a
[download] 100% of  941.84KiB in 00:00:00 at 1.71MiB/s     
[Merger] Merging formats into "Por qué ellos tienen éxito y tú no？ #desarrollopersonal #inspiracion #motivacion.mp4"
Deleting original file Por qué ellos tienen éxito y tú no？ #desarrollopersonal #inspiracion #motivacion.f140.m4a (pass -k to keep)
Deleting original file Por qué ellos tienen éxito y tú no？ #desarrollopersonal #inspiracion #motivacion.f788.mp4 (pass -k to keep)
[Vid

True

In [1]:
# Extract audio from video file
import os
from pydub import AudioSegment

# Configure ffmpeg paths
os.environ["PATH"] += ":/usr/local/bin"  # Add ffmpeg location to PATH
AudioSegment.ffmpeg = "/usr/local/bin/ffmpeg"
AudioSegment.ffprobe = "/usr/local/bin/ffprobe"

def extract_audio_from_video(video_file, output_audio_file):
    try:
        # Load the video file using name of the file
        if not os.path.exists(video_file):
            print(f"Error: The file {video_file} does not exist.")
            return
        if not video_file.lower().endswith(('.mp4', '.mkv', '.avi', '.mov')):
            print(f"Error: The file {video_file} is not a valid video format.")
            return
        # Load the video file
        print(f"Extracting audio from {video_file}...")
        video = AudioSegment.from_file(video_file)
        # Export the audio to the specified output file
        video.export(output_audio_file, format="mp3")
        print(f"Audio extracted successfully to {output_audio_file}")
    except Exception as e:
        print(f"An error occurred while extracting audio: {e}")

# Example usage
video_file = input('Enter video filename to extract audio from: ')
output_audio_file = video_file.rsplit('.', 1)[0] + '.mp3'
extract_audio_from_video(video_file, output_audio_file)
# Example: Extract audio from a video file
# output_audio_file = "example_audio.mp3"
# extract_audio_from_video(video_file, output_audio_file)
# Example: Extract audio from a video file
# video_file = input('Enter video filename to extract audio from: ')
# output_audio_file = video_file.rsplit('.', 1)[0] + '.mp3'

/opt/anaconda3/lib/python3.11/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


Extracting audio from himno.mp4...
Audio extracted successfully to himno.mp3
